In [ ]:
!pip install yfinance transformers accelerate datasets peft matplotlib seaborn torch psutil
!pip install -U peft

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import psutil
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Check System Resources
def check_resources():
    ram = psutil.virtual_memory().available / (1024**3)  # GB
    disk = psutil.disk_usage('/').free / (1024**3)  # GB
    print(f"Available RAM: {ram:.2f} GB, Available Disk: {disk:.2f} GB")
    if ram < 2 or disk < 10:
        raise RuntimeError("Insufficient resources for this operation.")
check_resources()

In [ ]:
# Model Configuration
BASE_MODEL = "EleutherAI/gpt-neo-1.3B"  # Base model without PEFT
REPORT_COLUMNS = [
    "Ticker", "Sector", "52-Week High", "52-Week Low",
    "Current Price", "Market Cap", "PE", "PB", "PEG", "ROC", "ROCE", "PH"
]

In [ ]:
# Load Base Model
def load_base_model(base_model_name):
    print("Loading base model (optimized for CPU)...")
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="cpu")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    return base_model, tokenizer

base_model, tokenizer = load_base_model(BASE_MODEL)

In [ ]:
# Fetch NIFTY 100 Stock Data
def fetch_nifty_100():
    print("Fetching NIFTY 100 data...")
    nifty_100_tickers = ["RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "HINDUNILVR.NS", "ICICIBANK.NS", "KOTAKBANK.NS", "SBIN.NS",
                         "BHARTIARTL.NS", "ITC.NS", "AXISBANK.NS", "BAJFINANCE.NS", "HCLTECH.NS", "MARUTI.NS", "SUNPHARMA.NS", "ADANIGREEN.NS",
                         "ADANIPORTS.NS", "ADANITRANS.NS", "AMBUJACEM.NS", "APOLLOHOSP.NS", "ASIANPAINT.NS", "AUROPHARMA.NS", "BAJAJ-AUTO.NS",
                         "BAJAJFINSV.NS", "BANDHANBNK.NS", "BANKBARODA.NS", "BEL.NS", "BERGEPAINT.NS", "BPCL.NS", "BRITANNIA.NS", "CIPLA.NS",
                         "COALINDIA.NS", "DABUR.NS", "DIVISLAB.NS", "DLF.NS", "DRREDDY.NS", "EICHERMOT.NS", "GAIL.NS", "GLAND.NS", "GODREJCP.NS",
                         "GRASIM.NS", "HAVELLS.NS", "HEROMOTOCO.NS", "HINDALCO.NS", "HINDPETRO.NS", "ICICIPRULI.NS", "INDIGO.NS", "INDUSINDBK.NS",
                         "IOC.NS", "IRCTC.NS", "JSWSTEEL.NS", "JUBLFOOD.NS", "L&TFH.NS", "LALPATHLAB.NS", "LICI.NS", "LT.NS", "LUPIN.NS",
                         "M&M.NS", "MCDOWELL-N.NS", "MINDTREE.NS", "MOTHERSUMI.NS", "NAUKRI.NS", "NMDC.NS", "NTPC.NS", "ONGC.NS", "PAGEIND.NS",
                         "PEL.NS", "PETRONET.NS", "PIIND.NS", "PNB.NS", "POWERGRID.NS", "PVR.NS", "RAMCOCEM.NS", "SBILIFE.NS", "SHREECEM.NS",
                         "SIEMENS.NS", "SRF.NS", "SRTRANSFIN.NS", "TATACHEM.NS", "TATACONSUM.NS", "TATAMOTORS.NS", "TATAPOWER.NS", "TATASTEEL.NS",
                         "TECHM.NS", "TITAN.NS", "TORNTPHARM.NS", "TORNTPOWER.NS", "TVSMOTOR.NS", "UBL.NS", "ULTRACEMCO.NS", "UPL.NS", "VEDL.NS",
                         "VOLTAS.NS", "WIPRO.NS", "ZEEL.NS", "ZYDUSLIFE.NS"]

    try:
        stocks_data = yf.download(nifty_100_tickers, period="1y", progress=False)
        return stocks_data
    except Exception as e:
        print(f"Error fetching stock data: {e}")
        return pd.DataFrame()
stocks_data = fetch_nifty_100()

In [ ]:
def process_stock_data(stocks_data, sector_mapping):
    print("Processing stock data...")
    stocks_summary = []

    if stocks_data.empty:
        print("No stock data available. Exiting processing.")
        return pd.DataFrame()  # Return empty DataFrame if no data

    for ticker in stocks_data.columns.levels[1]:
        try:
            ticker_data = stocks_data["Adj Close", ticker].dropna()
            if ticker_data.empty:
                print(f"No adjusted close data for {ticker}. Skipping.")
                continue

            high = ticker_data.max()
            low = ticker_data.min()
            current_price = ticker_data.iloc[-1]

            # Mock additional parameters
            date = ticker_data.index[-1]
            volume = stocks_data["Volume", ticker].iloc[-1] if "Volume" in stocks_data.columns.levels[0] else None
            sector = sector_mapping.get(ticker.split('.')[0], "Unknown")
            market_cap = 1_000_000  # Placeholder
            pe, pb, peg, roc, roce, ph = 15, 1.5, 1.2, 25, 30, 70  # Example placeholders

            stocks_summary.append({
                "Date": date, "Ticker": ticker, "Sector": sector,
                "52-Week High": high, "52-Week Low": low, "Current Price": current_price,
                "Volume": volume, "Market Cap": market_cap, "PE": pe, "PB": pb,
                "PEG": peg, "ROC": roc, "ROCE": roce, "PH": ph
            })
        except IndexError:
            print(f"IndexError for {ticker}. Skipping.")
        except KeyError:
            print(f"KeyError for {ticker}. Skipping.")

    return pd.DataFrame(stocks_summary)

In [ ]:
sector_mapping = {     "RELIANCE": "Energy",     "TCS": "IT",     "HDFCBANK": "Financials",
                  "INFY": "IT",     "HINDUNILVR": "Consumer Goods",     "ICICIBANK": "Financials",
                       "KOTAKBANK": "Financials",     "SBIN": "Financials",     "BHARTIARTL": "Telecom",
                       "ITC": "Consumer Goods",     "AXISBANK": "Financials",     "BAJFINANCE": "Financials",
                       "HCLTECH": "IT",     "MARUTI": "Automobile",     "SUNPHARMA": "Healthcare",     "ADANIGREEN": "Energy",
                       "ADANIPORTS": "Transportation",     "ADANITRANS": "Utilities",     "AMBUJACEM": "Materials",
                       "APOLLOHOSP": "Healthcare",     "ASIANPAINT": "Materials",     "AUROPHARMA": "Healthcare",
                       "BAJAJ-AUTO": "Automobile",     "BAJAJFINSV": "Financials",     "BANDHANBNK": "Financials",
                       "BANKBARODA": "Financials",     "BEL": "Industrials",     "BERGEPAINT": "Materials",     "BPCL": "Energy",
                       "BRITANNIA": "Consumer Goods",     "CIPLA": "Healthcare",     "COALINDIA": "Energy",
                       "DABUR": "Consumer Goods",     "DIVISLAB": "Healthcare",     "DLF": "Real Estate",     "DRREDDY": "Healthcare",
                       "EICHERMOT": "Automobile",     "GAIL": "Energy",     "GLAND": "Healthcare",     "GODREJCP": "Consumer Goods",
                       "GRASIM": "Materials",     "HAVELLS": "Consumer Goods",     "HEROMOTOCO": "Automobile",     "HINDALCO": "Materials",
                       "HINDPETRO": "Energy",     "ICICIPRULI": "Financials",     "INDIGO": "Transportation",     "INDUSINDBK": "Financials",
                       "IOC": "Energy",     "IRCTC": "Transportation",     "JSWSTEEL": "Materials",     "JUBLFOOD": "Consumer Goods",
                       "L&TFH": "Financials",     "LALPATHLAB": "Healthcare",     "LICI": "Financials",     "LT": "Industrials",
                       "LUPIN": "Healthcare",     "M&M": "Automobile",     "MCDOWELL-N": "Consumer Goods",     "MINDTREE": "IT",
                       "MOTHERSUMI": "Automobile",     "NAUKRI": "IT",     "NMDC": "Materials",     "NTPC": "Utilities",     "ONGC": "Energy",
                       "PAGEIND": "Consumer Goods",     "PEL": "Financials",     "PETRONET": "Energy",     "PIIND": "Materials",
                       "PNB": "Financials",     "POWERGRID": "Utilities",     "PVR": "Consumer Services",     "RAMCOCEM": "Materials",
                       "SBILIFE": "Financials",     "SHREECEM": "Materials",     "SIEMENS": "Industrials",     "SRF": "Materials",
                       "SRTRANSFIN": "Financials", "TATACHEM": "Materials", "TATACONSUM": "Consumer Goods", "TATAMOTORS": "Automobile",
                       "TATAPOWER": "Utilities", "TATASTEEL": "Materials", "TECHM": "IT", "TITAN": "Consumer Goods", "TORNTPHARM": "Healthcare",
                       "TORNTPOWER": "Utilities", "TVSMOTOR": "Automobile", "UBL": "Consumer Goods", "ULTRACEMCO": "Materials", "UPL": "Materials",
                       "VEDL": "Materials", "VOLTAS": "Consumer Goods", "WIPRO": "IT", "ZEEL": "Consumer Services", "ZYDUSLIFE": "Healthcare" }
stocks_summary = process_stock_data(stocks_data, sector_mapping)

In [ ]:
# Filter Stocks Based on Criteria
def filter_stocks(stocks_summary):
    print("Filtering stocks based on optimization criteria...")
    filtered_stocks = stocks_summary[
        (stocks_summary["PE"] < 20) &
        (stocks_summary["PB"] < 2) &
        (stocks_summary["PEG"] < 2) &
        (stocks_summary["ROC"] > 20) &
        (stocks_summary["ROCE"] > 20) &
        (stocks_summary["PH"] > 60)
    ]
    return filtered_stocks

filtered_stocks = filter_stocks(stocks_summary)


In [ ]:
def generate_recommendations(base_model, tokenizer, stock_data):
    print("Generating recommendations...")

    # Create the pipeline without specifying a device
    pipe = pipeline("text-generation", model=base_model, tokenizer=tokenizer)

    recommendations = []
    for _, row in stock_data.iterrows():
        prompt = (
            f"Provide investment advice for {row['Ticker']} in the {row['Sector']} sector. "
            f"52-week high: {row['52-Week High']}, low: {row['52-Week Low']}, "
            f"current price: {row['Current Price']}, PE: {row['PE']}, PB: {row['PB']}."
        )
        # Generate recommendations
        result = pipe(prompt, max_length=100, num_return_sequences=1)
        recommendations.append(result[0]["generated_text"])

    # Add recommendations to the DataFrame
    stock_data["Recommendations"] = recommendations
    return stock_data

# Ensure 'filtered_stocks' is not empty before running
if not filtered_stocks.empty:
    recommendations = generate_recommendations(base_model, tokenizer, filtered_stocks)
else:
    print("No stocks available for recommendation.")

In [ ]:
# Visualize Performance
def visualize_performance(filtered_stocks):
    print("Visualizing performance trends...")
    plt.figure(figsize=(12, 8))
    sns.lineplot(data=filtered_stocks, x="Ticker", y="Current Price", hue="Sector", marker="o")
    plt.title("Performance Trends of Selected Stocks")
    plt.xticks(rotation=45)
    plt.show()

def visualize_top_10(filtered_stocks):
    top_10 = filtered_stocks.nlargest(10, "Current Price")
    plt.figure(figsize=(12, 8))
    sns.barplot(data=top_10, x="Ticker", y="Current Price", hue="Sector")
    plt.title("Top 10 Stocks by Current Price")
    plt.show()

def visualize_sector_distribution(filtered_stocks):
    plt.figure(figsize=(12, 8))
    sns.countplot(data=filtered_stocks, x="Sector", palette="cool")
    plt.title("Sector Distribution of Selected Stocks")
    plt.xticks(rotation=45)
    plt.show()

# Final Visualization Calls
if not filtered_stocks.empty:
    visualize_performance(filtered_stocks)
    visualize_top_10(filtered_stocks)
    visualize_sector_distribution(filtered_stocks)
    print("Final Recommendations:")
    print(recommendations[["Ticker", "Sector", "Recommendations"]])
else:
    print("No data to visualize.")

In [ ]:
from sklearn.model_selection import train_test_split

def predict_future_prices(filtered_stocks, stocks_data, days=30):
    print("Predicting future prices for top 10 performing stocks...")

    top_10 = filtered_stocks.nlargest(10, "Current Price")
    predictions = {}

    for ticker in top_10["Ticker"]:
        try:
            # Extract historical data for the ticker
            ticker_data = stocks_data["Adj Close", ticker].dropna()

            # Create lagged features
            df = pd.DataFrame({"Price": ticker_data.values})
            for lag in range(1, 6):  # Create 5 lagged features
                df[f"Lag_{lag}"] = df["Price"].shift(lag)
            df = df.dropna()  # Drop rows with missing values

            # Prepare data for training
            X = df.drop(columns=["Price"]).values
            y = df["Price"].values
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Train a RandomForestRegressor
            model = RandomForestRegressor(n_estimators=100, random_state=42)
            model.fit(X_train, y_train)

            # Predict future prices
            last_known_features = X[-1]  # Start with the most recent data
            future_prices = []

            for _ in range(days):
                next_price = model.predict([last_known_features])[0]
                future_prices.append(next_price)

                # Update features for the next prediction
                last_known_features = np.roll(last_known_features, -1)
                last_known_features[-1] = next_price

            predictions[ticker] = future_prices

        except Exception as e:
            print(f"Error processing {ticker}: {e}")
            continue

    return predictions


In [ ]:
def visualize_future_predictions(future_predictions, stocks_data, days=30):
    print("Visualizing historical and predicted prices...")

    plt.figure(figsize=(14, 8))

    for i, (ticker, future_prices) in enumerate(future_predictions.items()):
        ticker_data = stocks_data["Adj Close", ticker].dropna()
        historical_prices = ticker_data.values

        # Combine historical and predicted prices
        time_historical = np.arange(len(historical_prices))
        time_future = np.arange(len(historical_prices), len(historical_prices) + days)

        plt.plot(time_historical, historical_prices, label=f"{ticker} - Historical")
        plt.plot(time_future, future_prices, linestyle="--", label=f"{ticker} - Predicted")

    plt.title("Historical and Predicted Prices for Top 10 Performing Stocks")
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.legend()
    plt.show()


In [ ]:
# Visualization 2: Individual Stock Trends
def visualize_individual_stocks(future_predictions, stocks_data, days=30):
    print("Creating individual stock trend subplots...")
    num_stocks = len(future_predictions)
    fig, axes = plt.subplots(num_stocks, 1, figsize=(10, 5 * num_stocks), sharex=True)

    for ax, (ticker, future_prices) in zip(axes, future_predictions.items()):
        ticker_data = stocks_data["Adj Close", ticker].dropna()
        historical_prices = ticker_data.values

        # Combine historical and predicted prices
        all_prices = np.concatenate([historical_prices, future_prices])
        time = np.arange(len(all_prices))

        ax.plot(time[:len(historical_prices)], historical_prices, label="Historical", color="blue")
        ax.plot(time[len(historical_prices):], future_prices, linestyle="--", label="Predicted", color="orange")
        ax.set_title(f"{ticker} - Historical vs. Predicted Prices")
        ax.set_ylabel("Price")
        ax.legend()

    plt.xlabel("Days")
    plt.tight_layout()
    plt.show()


In [ ]:
# Final Calls to New Visualizations
if not filtered_stocks.empty:
    future_predictions = predict_future_prices(filtered_stocks, stocks_data)
    visualize_future_predictions(future_predictions, stocks_data)
    visualize_individual_stocks(future_predictions, stocks_data)
else:
    print("No data available for future price prediction.")

In [ ]:
from google.colab import drive

In [ ]:
import joblib

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Define the folder path in Google Drive
google_drive_folder = "/content/drive/MyDrive/Colab Notebooks/Stock_Recommendation"

In [ ]:
import os
from google.colab import files

In [ ]:
def save_model_and_config(model, tokenizer, sector_mapping, folder_path=google_drive_folder, file_name="stock_recommend.pkl"):
    file_path = os.path.join(folder_path, file_name)
    print(f"Saving model and configuration to {file_path}...")

    # Save the model and tokenizer
    model_path = os.path.join(folder_path, "model")
    model.save_pretrained(model_path)
    print(f"Base model saved to {model_path}")

    tokenizer_path = os.path.join(folder_path, "tokenizer")
    tokenizer.save_pretrained(tokenizer_path)
    print(f"Tokenizer saved to {tokenizer_path}")

    # Save the sector mapping
    joblib.dump({
        "sector_mapping": sector_mapping
    }, file_path)
    print(f"Sector mapping and additional configurations saved to {file_path}")

    print("Model and configuration saved successfully.")

    # Download the files
    print("Downloading files...")
    files.download(file_path)
    for filename in os.listdir(model_path):
        files.download(os.path.join(model_path, filename))
    for filename in os.listdir(tokenizer_path):
        files.download(os.path.join(tokenizer_path, filename))
    print("Files downloaded successfully.")


In [ ]:
save_model_and_config(base_model, tokenizer, sector_mapping)